In [10]:
#coding:utf-8
import ctypes
import numpy as np
import scipy.interpolate as sci_interpolate
import time
import sys
from multiprocessing import Process,Pool
import scipy.optimize as sciop
import scipy.constants as sciconsts
import scipy.interpolate as sci_interpolate
import scipy.integrate as sci_integrate
'''parameters'''
def chirp_mass(m1,m2):
    return np.power(m1*m2,0.6)/(np.power(m1+m2,0.2))
def solvem(mc,smr):
    a=mc/np.power(smr,0.6)
    b=smr*np.power(a,2.0)
    m1=(a+np.sqrt(np.power(a,2.0)-4.0*b))/2.0
    m2=(a-np.sqrt(np.power(a,2.0)-4.0*b))/2.0
    return np.array((m1,m2))
def sym_mass_ratio(m1,m2):
    return m1*m2/(np.power(m1+m2,2.0))
'''lisa noise curve in rpsd unit N2A5:laserpower 2w,40cm telescope,armlength 5e6 km'''
def S_gal_N2A5(f):
    if f>=1.0e-5 and f<1.0e-3:
        return np.power(f,-2.3)*np.power(10,-44.62)*20.0/3.0
    if f>=1.0e-3 and f<np.power(10,-2.7):
        return np.power(f,-4.4)*np.power(10,-50.92)*20.0/3.0
    if f>=np.power(10,-2.7) and f<np.power(10,-2.4):
        return np.power(f,-8.8)*np.power(10,-62.8)*20.0/3.0
    if f>=np.power(10,-2.4) and f<=0.01:
        return np.power(f,-20.0)*np.power(10,-89.68)*20.0/3.0
    if f>0.01 or f<1.0e-5:
        return 0
def S_n_lisa(f):
    m1=5.0e9
    m2=sciconsts.c*0.41/m1/2.0
    return 20.0/3.0*(1+np.power(f/m2,2.0))*(4.0*(9.0e-30/np.power(2*sciconsts.pi*f,4.0)*(1+1.0e-4/f))+2.96e-23+2.65e-23)/np.power(m1,2.0)+S_gal_N2A5(f)
    
    

In [ ]:
#载入库
ll=ctypes.cdll.LoadLibrary
bin=ll("./bingas.so")
dd=ctypes.c_double
#consts setting SI unit
timescale=1.0
m_sun=1.9891e30
years=365*24*3600.0
t_scale=years*timescale
t_gas=1000.0*years

#气体信号构建
m1_g=10.0*m_sun
m2_g=10.0*m_sun
gas_result=filenamecostruct(chirp_mass(m_1g,m_2g),sym_mass_ratio(m_1g,m_2g),0.02,True)
bin.maincal(dd(chirp_mass(m_1g,m_2g)),dd(sym_mass_ratio(m_1g,m_2g)),dd(0.002),dd(t_scale),dd(t_gas),ctypes.c_char_p(gas_result))
gas_data=np.load(gas_result)
tdata=gas_data.T[3]
phasedata=gas_data.T[2]
rdata=gas_data.T[0]*sciconsts.astronomical_unit
r_t_gas=sci_interpolate.interp1d(tdata,rdata)
phi_t_gas=sci_interpolate.interp1d(tdata,phasedata)
#注意 这里我们已经不能得到频域的积分上下界，故使用0和np.inf代替
A=sci_integrate.quad(lambda x:4np.power(1.0/r_t_gas(x)*np.cos(phi_t_gas(x)),2.0)/S_n_lisa(x),0,np.inf,limit=1500,epsabs=0.0005) 
Aval=np.sqrt(A[0]) 
Aerr=0.5/AvalA[1] 
Aval,Aerr

'vac40.000000-0.250000.npy'

In [9]:
np.load("/Users/shyn/Desktop/test.npy")

array([[  1.99999835e-03,   1.86836052e-03,   6.55196501e+00,
          1.21313918e+01],
       [  1.99999834e-03,   1.86836324e-03,   6.82056165e+00,
          2.42627836e+01],
       [  1.99999833e-03,   1.86836673e-03,   7.10818299e+00,
          3.72615724e+01],
       ..., 
       [  1.99999565e-03,   1.86837051e-03,   8.60094887e+01,
          3.59734132e+03],
       [  1.99999565e-03,   1.86836972e-03,   8.60685542e+01,
          3.60000000e+03],
       [  1.99999565e-03,   1.86836972e-03,   8.60685542e+01,
          3.60000000e+03]])